# Jira SQL to generate the file
project = "BPD" AND labels = Release_Notes AND fixversion = 12-13-2024 ORDER BY key DESC, created DESC

In [ ]:
# Import necessary libraries
import os
import pandas as pd
from openai import OpenAI
from datetime import datetime

# Get openai Key from os settings
open_ai_api_key = os.getenv('OPENAI_API')

# Set your OpenAI API key
client = OpenAI(
    api_key = open_ai_api_key
)

# Get and format todays date to include in the html body of the update.
def get_day_suffix(day):
    if 4 <= day <= 20 or 24 <= day <= 30:
        return "th"
    else:
        return ["st", "nd", "rd"][day % 10 - 1]

# Get today's date
today = datetime.today()

# Extract day, month, and year
day = today.day
month = today.strftime("%B")  # Full month name
year = today.year

# Format the date
formatted_date = f"{month} {day}{get_day_suffix(day)}, {year}"

# Get day of week
day_of_week = today.strftime("%A") 

# Function to create a single-sentence release note using OpenAI's ChatGPT
def create_release_note_for_story(title, description, labels):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {
        "role": "user", 
        "content": 
            f"Create a single-sentence release note for the following issue:\n\n"
            f"Title: {title}\nDescription: {description}.\n\n"
            f"If it is a bug and not labeled with 'global' then make sure we include the note 'for some systems. This rule only applies to non whiteboard issues.' "
            f"so people don't think it was broken in their instance as well."
            f"Here are the labels for the issue {labels} \n\n"
            f"The note should be in plane language and be as short as possible. Assume this is a non-technical audience.\n"
            f"Please avoid rephrasing or expanding on my statements. Just provide the specific term or concept I'm asking for without additional context or suggestions.\n"
            f"If the title has WB then it is for whiteboard.\n"
            f"DO NOT hullucinate.\n"
            f"DO NOT use any abbreviations, if you see VI make it View Idea, etc..\n\n"
            f"DO NOT include anyones name or any business names.\n"
            f"DO NOT inclue ``` or html anywhere in the response.\n"
            f"ONLY return valid HTML, nothing outside of the <ul></ul> elements.\n"
            f"Here are 5 examples of really well written release notes:\n"
            f"<strong>Addressed Confusion with Team Workspace Submit-</strong> We removed the active Submit button from the Team Workspace page when Submission is turned off to reduce confusion.\n"
            f"<strong>View Idea 3 Dropdown Transparency Fix-</strong> We fixed a transparency issue within a dropdown on View Idea 3.\n"
            f"<strong>Restored Unordered List Button-</strong> We fixed the unresponsive Unordered List button in the Initiative-level Rich Text Editor 2.0 that had affected some systems.\n"
            f"<strong>Blue Diamond Gate Object-</strong> We updated the default Gate object to display a blue diamond emoji instead of an orange diamond.\n"
            f"<strong>Toggle Logic for Table Tool-</strong> We updated the logic in the Whiteboard Left Toolbar to ensure that the Tables feature is displayed when enabled."
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=500
    )
    return response.choices[0].message.content


# Function to categorize release notes into sections using OpenAI's ChatGPT
def categorize_release_notes(release_notes):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": 
            f"Categorize the following release notes into sections like:" 
            f"'General bug fixes and enhancements', 'Project Room', 'Whiteboard', and 'Hackathon':\n\n"
            f"{release_notes}.\n\n"
            f"Make sure to return the list in the correct HTML format:\n\n"
            f"<p><strong>Category Title</strong></p>"
            f"<ul>"
            f"<li>Release note.</li>"
            f"<li>Release note</li>"
            f"<li>...</li>"
            f"</ul>"
            f"DO NOT inclue ``` or html anywhere in the response.\n"
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=4096
    )
    return response.choices[0].message.content
    

# Main function to process the release stories from a CSV file and generate release notes
def generate_release_notes_from_csv(csv_file_path):
    # Read the CSV file into a pandas DataFrame
    issues_df = pd.read_csv(csv_file_path)
    
    release_notes = ""
    for index, row in issues_df.iterrows():
        title = row['Summary']
        description = row.get('Description', 'No description provided.')
        labels = row.get('Labels')
        release_note = create_release_note_for_story(title, description, labels)
        release_notes += f"<li>{release_note}</li>\n"
    
    categorized_notes = categorize_release_notes(release_notes)
    
    formatted_release_notes = f"""
<p>
  Our&nbsp;latest<span style="box-sizing: border-box;">&nbsp;</span><span style="box-sizing: border-box;">product release took effect</span><span style="box-sizing: border-box;">&nbsp;</span><strong style="box-sizing: border-box; font-weight: 600;">{day_of_week}</strong><strong style="box-sizing: border-box; font-weight: 600;">, {formatted_date}</strong><strong style="box-sizing: border-box; font-weight: 600;">.<span style="box-sizing: border-box;">&nbsp;</span></strong>This
  post may be different from the release notes received via email.
</p>
<p>
  <span style="box-sizing: border-box;">To read Brightidea's complete documentation, you can visit the </span><a style="box-sizing: border-box; background-color: transparent; color: #034678; text-decoration: none;" href="https://support.brightidea.com/hc/en-us/sections/200825397-Product-Release-Notes">Product Release Notes</a><span style="box-sizing: border-box;">&nbsp;forum in the Support Portal.</span>
</p>
{categorized_notes}
"""
    return formatted_release_notes

# Example usage
home_directory = os.path.expanduser("~")
desktop_directory = os.path.join(home_directory, "Desktop")
folder_name = "ReleaseNotes"  # Replace with your folder name
file_name = "brightidea.jira.com.csv"  # Replace with your file name
csv_file_path = os.path.join(desktop_directory, folder_name, file_name)

release_notes_html = generate_release_notes_from_csv(csv_file_path)
print(release_notes_html)